In [ ]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Input, Add, Concatenate, GlobalAveragePooling2D, Dense, Reshape
from tensorflow.keras.layers import ZeroPadding2D,MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.initializers import glorot_uniform

dataset_dir = '/kaggle/input/ziptrain/mini proj'
dataset_dir2 = '/kaggle/input/ziptest/validation set'
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32


train_generator = datagen.flow_from_directory(
    dataset_dir,            
    target_size=(224, 224),  
    batch_size=batch_size,
    class_mode='categorical')


validation_generator = datagen.flow_from_directory(
    dataset_dir2,            
    target_size=(224, 224),  
    batch_size=batch_size,
    class_mode='categorical')

def identity_block(X, f, filters, stage, block):
   
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = ReLU()(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = ReLU()(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a ReLU activation
    X = Add()([X, X_shortcut])
    X = ReLU()(X)

    return X

def convolutional_block(X, f, filters, stage, block, s = 2):
    
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = ReLU()(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = ReLU()(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s, s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a ReLU activation
    X = Add()([X, X_shortcut])
    X = ReLU()(X)
    
    return X

def ResNet50(input_shape=(224, 224, 3), classes=1000):
   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = ReLU()(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = GlobalAveragePooling2D()(X)

    # output layer
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

# Create the model
model = ResNet50(input_shape=(224, 224, 3), classes=3)

# Define the optimizer
optimizer1 = Adam(learning_rate=0.001)
optimizer2 = SGD(learning_rate=0.01)


model.compile(optimizer=optimizer1, loss='categorical_crossentropy', metrics=['accuracy'])

train_accs = []
train_losses = []
val_accs = []
val_losses = []

for epoch in tqdm(range(10), desc="Training Epochs"):
    history = model.fit(train_generator, epochs=1, validation_data=validation_generator)
    
    train_accs.append(history.history['accuracy'][0])
    train_losses.append(history.history['loss'][0])
    val_accs.append(history.history['val_accuracy'][0])
    val_losses.append(history.history['val_loss'][0])


# Compile the model with the second optimizer
model.compile(optimizer=optimizer2, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with the second optimizer
for epoch in tqdm(range(90), desc="Training Epochs"):
    history2 = model.fit(train_generator, epochs=1, validation_data=validation_generator)
    
    train_accs.append(history2.history['accuracy'][0])
    train_losses.append(history2.history['loss'][0])
    val_accs.append(history2.history['val_accuracy'][0])
    val_losses.append(history2.history['val_loss'][0])

# Save the model
model.save('resnet50_model.h5')


In [10]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
loaded_model = tf.keras.models.load_model('resnet50_model.h5')
your_image_path = "/kaggle/input/prostate-pred/0004.png"
img = image.load_img(your_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Convert single image to batch of size 1

# Preprocess the image
img_array = img_array / 255.0  # Normalize pixel values to [0, 1]

# Make predictions
predictions = loaded_model.predict(img_array)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
[[3.1504896e-04 9.9967653e-01 8.3557670e-06]]
